### Question:
You have a dataset represented as a list of dictionaries. Each entry contains keys: "region", "product", "sales", and "date" (a string in YYYY-MM-DD format). Write a Python program to:

1. Parse the dataset and convert the dates into datetime objects for easy manipulation.
2. Identify the month with the highest total sales for each region.
3. Perform a time-series trend analysis: calculate a 3-month rolling average of sales for each product.
4. Group the data by product and compute the percentage growth/decline in sales compared to the previous month.

In [1]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [2]:
# Sample dataset
sales_data = [
    {"region": "North", "product": "A", "sales": 120, "date": "2024-01-15"},
    {"region": "North", "product": "A", "sales": 200, "date": "2024-02-10"},
    {"region": "North", "product": "A", "sales": 150, "date": "2024-03-12"},
    {"region": "South", "product": "B", "sales": 300, "date": "2024-01-20"},
    {"region": "South", "product": "B", "sales": 250, "date": "2024-02-15"},
    {"region": "South", "product": "B", "sales": 180, "date": "2024-03-18"},
    {"region": "East", "product": "C", "sales": 400, "date": "2024-01-25"},
    {"region": "East", "product": "C", "sales": 350, "date": "2024-02-12"},
    {"region": "East", "product": "C", "sales": 450, "date": "2024-03-10"},
]

In [3]:
# Step 1: Convert dates to datetime objects and load into a DataFrame
df = pd.DataFrame(sales_data)
df['date'] = pd.to_datetime(df['date'])

In [4]:
# Step 2: Identify the month with the highest sales for each region
df['month'] = df['date'].dt.to_period('M')
region_monthly_sales = df.groupby(['region', 'month'])['sales'].sum().reset_index()
highest_sales_by_region = region_monthly_sales.loc[
    region_monthly_sales.groupby('region')['sales'].idxmax()
]

In [5]:
# Step 3: 3-month rolling average sales for each product
df = df.sort_values('date')
product_monthly_sales = df.groupby(['product', 'month'])['sales'].sum().reset_index()
product_monthly_sales['rolling_avg_3m'] = product_monthly_sales.groupby('product')['sales'].transform(lambda x: x.rolling(3, min_periods=1).mean())

In [6]:
# Step 4: Percentage growth/decline in sales compared to the previous month
product_monthly_sales['pct_change'] = product_monthly_sales.groupby('product')['sales'].pct_change() * 100

In [7]:
# Results
print("Highest sales month for each region:")
print(highest_sales_by_region, end="\n\n")

Highest sales month for each region:
  region    month  sales
2   East  2024-03    450
4  North  2024-02    200
6  South  2024-01    300



In [8]:
print("3-month rolling average sales per product:")
print(product_monthly_sales[['product', 'month', 'sales', 'rolling_avg_3m']], end="\n\n")

3-month rolling average sales per product:
  product    month  sales  rolling_avg_3m
0       A  2024-01    120      120.000000
1       A  2024-02    200      160.000000
2       A  2024-03    150      156.666667
3       B  2024-01    300      300.000000
4       B  2024-02    250      275.000000
5       B  2024-03    180      243.333333
6       C  2024-01    400      400.000000
7       C  2024-02    350      375.000000
8       C  2024-03    450      400.000000



In [9]:
print("Percentage growth/decline in sales by product:")
print(product_monthly_sales[['product', 'month', 'sales', 'pct_change']])

Percentage growth/decline in sales by product:
  product    month  sales  pct_change
0       A  2024-01    120         NaN
1       A  2024-02    200   66.666667
2       A  2024-03    150  -25.000000
3       B  2024-01    300         NaN
4       B  2024-02    250  -16.666667
5       B  2024-03    180  -28.000000
6       C  2024-01    400         NaN
7       C  2024-02    350  -12.500000
8       C  2024-03    450   28.571429
